In [ ]:
# 코랩과 AWS를 연동하여
# 1.ciga 테이블을 만들고
# 2.MySQL 구문을 활용하여, 종목 선정 기준용 데이터 프레임을 만든다.

In [ ]:
# 1. ciga_df 만들기
# 우리조는 자체적으로 수정한 DB_SQLSTK_FULL 테이블과 ciga라는 새로운 테이블을 만들어 사용하였다.
# 여기서는 ciga 테이블을 만들것이다.

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as ms

In [ ]:
# 주가 정보를 스크랩하는 파이썬 라이브러리
!pip install pykrx

     |████████████████████████████████| 81 kB 9.9 MB/s 
     |████████████████████████████████| 60 kB 8.0 MB/s 
     |████████████████████████████████| 251 kB 37.9 MB/s 


In [ ]:
# 유가 증권시장의 주식 정보를 스크래핑
from pykrx import stock

In [ ]:
# '20181231'일자 종목별 시가총액 가져오기
df = stock.get_market_cap_by_ticker("20181231")
display(df)

The date you entered 20181231 seems to be a holiday. PYKRX changes the date parameter to 20181228 to query the requested information.


,종가,시가총액,거래량,거래대금,상장주식수
티커,,,,,
005930,38700,231030584685000,9900267,382575637240,5969782550
000660,60500,44044143082500,1647462,100586317600,728002365
068270,222500,27913989592500,1098718,243957028500,125456133
005935,31750,26126652725000,863411,27222988450,822886700
207940,386500,25572772500000,256133,99632653500,66165000
...,...,...,...,...,...
179720,188,2115000000,1,188,11250000
200350,1075,2079774550,6799,7363715,1934674
135270,315,1726282845,155,64841,5480263


In [ ]:
# 인덱스 재설정
df.reset_index(inplace=True)
df

,티커,종가,시가총액,거래량,거래대금,상장주식수
0,005930,38700,231030584685000,9900267,382575637240,5969782550
1,000660,60500,44044143082500,1647462,100586317600,728002365
2,068270,222500,27913989592500,1098718,243957028500,125456133
3,005935,31750,26126652725000,863411,27222988450,822886700
4,207940,386500,25572772500000,256133,99632653500,66165000
...,...,...,...,...,...,...
2375,179720,188,2115000000,1,188,11250000
2376,200350,1075,2079774550,6799,7363715,1934674
2377,135270,315,1726282845,155,64841,5480263
2378,189540,650,1570393500,8101,5810650,2415990


In [ ]:
# '티커'열 데이터 문자형으로 변환
df['티커'] = list( map(str, df['티커']) )

In [ ]:
# '티커'열 데이터프레임 문자형으로 변환
df['티커'] = df['티커'].astype(str)

In [ ]:
df.티커 = df.티커.astype(str)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   티커      2380 non-null   object
 1   종가      2380 non-null   int64 
 2   시가총액    2380 non-null   int64 
 3   거래량     2380 non-null   int64 
 4   거래대금    2380 non-null   int64 
 5   상장주식수   2380 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 111.7+ KB


In [ ]:
type(df['티커'][0])

str

In [ ]:
# df를 'pykrx_20181231.csv'로 저장
df.to_csv('pykrx_20181231.csv', encoding='cp949')

In [ ]:
# csv 파일을 df2로 불러오기(확인차 실행)
# df2= pd.read_csv('pykrx_20181231.csv', encoding='cp949', engine='python', index_col=0)
# df2

In [ ]:
# PyMySQL 설치
! pip install PyMySQL

     |████████████████████████████████| 43 kB 2.4 MB/s 


In [ ]:
# 오류시, PyMySQL 설치
! python3 -m pip install PyMySQL[rsa]
import pymysql.cursors

     |████████████████████████████████| 3.0 MB 31.3 MB/s 


In [ ]:
# 오류난다면 이 부분 추가
! pip3 install crytography

ERROR: Could not find a version that satisfies the requirement crytography (from versions: 0.0.1, 1.0.0)
ERROR: No matching distribution found for crytography


In [ ]:
# MySQL에 연결 설정
connection = pymysql.connect(
    host='3.36.51.240', #EC2 아이피
    user='USER_SQLSTK', #데이터베이스 사용자
    password='1qaz2wsx', #데이터베이스 암호
    database='DB_SQLSTK', #데이터베이스 이름
    charset='utf8', #언어 설정 
    cursorclass=pymysql.cursors.DictCursor #조회결과를 dictionary로 리턴
)

In [ ]:
connection

In [ ]:
# 연결 실행
cursor = connection.cursor()

In [ ]:
cursor

In [ ]:
# ciga table import해야하는데
# ciga table이 이미 존재할시에는(OperationalError: (1050, "Table 'ciga' already exists"))
# 아래 세줄을 주석 처리 해제하고 ciga table을 drop한다
# sql = """
# DROP TABLE `ciga`;
# """

In [ ]:
# ciga table이 이미 존재할시에는
# 아래 한줄을 주석 처리 해제하고 ciga table을 drop한다
# cursor.execute(sql)

0

In [ ]:
# ciga table 만들기
# 실행할 SQL
sql = ''
sql = """
CREATE TABLE `ciga`
(
  `STK_CD` varchar(40) CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci NOT NULL COMMENT '종목코드',
  `C_PRC` decimal(18,3) DEFAULT NULL COMMENT '종가',
  `CIGA_SUM` decimal(18,3) DEFAULT NULL COMMENT '시가총액',
  `VOL` decimal(18,3) DEFAULT NULL COMMENT '거래량',
  `AMOUNT` decimal(18,3) DEFAULT NULL COMMENT '거래대금',
  `STOCK_CNT` decimal(18,3) DEFAULT NULL COMMENT '상자주식수',
  PRIMARY KEY (`STK_CD`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci COMMENT='시가';
"""

In [ ]:
# SQL 쿼리를 실행할 준비를 함
cursor.execute(sql)

0

In [ ]:
# SQL 쿼리를 실행하고 결과를 result에 저장
res = cursor.fetchall()
# SQL 쿼리 실행 결과 출력
res

()

In [ ]:
sql = ''
sql = '''
DESC ciga;
'''

In [ ]:
cursor.execute(sql)

6

In [ ]:
res = cursor.fetchall()
res

[{'Default': None,
  'Extra': '',
  'Field': 'STK_CD',
  'Key': 'PRI',
  'Null': 'NO',
  'Type': 'varchar(40)'},
 {'Default': None,
  'Extra': '',
  'Field': 'C_PRC',
  'Key': '',
  'Null': 'YES',
  'Type': 'decimal(18,3)'},
 {'Default': None,
  'Extra': '',
  'Field': 'CIGA_SUM',
  'Key': '',
  'Null': 'YES',
  'Type': 'decimal(18,3)'},
 {'Default': None,
  'Extra': '',
  'Field': 'VOL',
  'Key': '',
  'Null': 'YES',
  'Type': 'decimal(18,3)'},
 {'Default': None,
  'Extra': '',
  'Field': 'AMOUNT',
  'Key': '',
  'Null': 'YES',
  'Type': 'decimal(18,3)'},
 {'Default': None,
  'Extra': '',
  'Field': 'STOCK_CNT',
  'Key': '',
  'Null': 'YES',
  'Type': 'decimal(18,3)'}]

In [ ]:
sql = ''
sql = '''
  SHOW TABLES;
'''

In [ ]:
cursor.execute(sql)

10

In [ ]:
# df를 dictionary('split') 형태로 전환 -> 'columns', 'data'로 나뉨
dict_ciga = df.to_dict('split')
dict_ciga

{'columns': ['티커', '종가', '시가총액', '거래량', '거래대금', '상장주식수'],
 'data': [['005930',
   38700,
   231030584685000,
   9900267,
   382575637240,
   5969782550],
  ['000660', 60500, 44044143082500, 1647462, 100586317600, 728002365],
  ['068270', 222500, 27913989592500, 1098718, 243957028500, 125456133],
  ['005935', 31750, 26126652725000, 863411, 27222988450, 822886700],
  ['207940', 386500, 25572772500000, 256133, 99632653500, 66165000],
  ['005380', 118500, 25319680159500, 398062, 47289560500, 213668187],
  ['051910', 347000, 24495543021000, 107893, 37503088000, 70592343],
  ['017670', 269500, 21760969114500, 163072, 43987800500, 80745711],
  ['015760', 33100, 21249010948700, 1229000, 40846690500, 641964077],
  ['005490', 243000, 21186400905000, 130754, 31720146000, 87186835],
  ['035420', 122000, 20107234190000, 322665, 39344095000, 164813395],
  ['028260', 105500, 20012299536500, 185250, 19559467000, 189690043],
  ['105560', 46500, 19442186470500, 831292, 38509592150, 418111537],
  ['05555

In [ ]:
# 'data'를 ciga_ls로 설정
ciga_ls = dict_ciga['data']
ciga_ls

[['005930', 38700, 231030584685000, 9900267, 382575637240, 5969782550],
 ['000660', 60500, 44044143082500, 1647462, 100586317600, 728002365],
 ['068270', 222500, 27913989592500, 1098718, 243957028500, 125456133],
 ['005935', 31750, 26126652725000, 863411, 27222988450, 822886700],
 ['207940', 386500, 25572772500000, 256133, 99632653500, 66165000],
 ['005380', 118500, 25319680159500, 398062, 47289560500, 213668187],
 ['051910', 347000, 24495543021000, 107893, 37503088000, 70592343],
 ['017670', 269500, 21760969114500, 163072, 43987800500, 80745711],
 ['015760', 33100, 21249010948700, 1229000, 40846690500, 641964077],
 ['005490', 243000, 21186400905000, 130754, 31720146000, 87186835],
 ['035420', 122000, 20107234190000, 322665, 39344095000, 164813395],
 ['028260', 105500, 20012299536500, 185250, 19559467000, 189690043],
 ['105560', 46500, 19442186470500, 831292, 38509592150, 418111537],
 ['055550', 39600, 18778303645200, 849588, 33553303570, 474199587],
 ['012330', 190000, 18495333970000,

In [ ]:
# 데이터를 문자열 '' 형태로 변환
for i in range(len(ciga_ls)):
  ls = ciga_ls[i]
  for j in range(len(ls)):
    ciga_ls[i][j] = str(ls[j]).replace(' ', '')

In [ ]:
ciga_ls

[['005930',
  '38700',
  '231030584685000',
  '9900267',
  '382575637240',
  '5969782550'],
 ['000660', '60500', '44044143082500', '1647462', '100586317600', '728002365'],
 ['068270',
  '222500',
  '27913989592500',
  '1098718',
  '243957028500',
  '125456133'],
 ['005935', '31750', '26126652725000', '863411', '27222988450', '822886700'],
 ['207940', '386500', '25572772500000', '256133', '99632653500', '66165000'],
 ['005380', '118500', '25319680159500', '398062', '47289560500', '213668187'],
 ['051910', '347000', '24495543021000', '107893', '37503088000', '70592343'],
 ['017670', '269500', '21760969114500', '163072', '43987800500', '80745711'],
 ['015760', '33100', '21249010948700', '1229000', '40846690500', '641964077'],
 ['005490', '243000', '21186400905000', '130754', '31720146000', '87186835'],
 ['035420', '122000', '20107234190000', '322665', '39344095000', '164813395'],
 ['028260', '105500', '20012299536500', '185250', '19559467000', '189690043'],
 ['105560', '46500', '194421864

In [ ]:
len(ciga_ls[0])

6

In [ ]:
# ciga table에 df(ciga_ls) 값을 저장
# 표 제목은 ciga에 내용은 df
sql = ''
sql = '''
        INSERT INTO ciga VALUES(
          %s,%s,%s,%s,%s,%s
        );
      '''

In [ ]:
cursor.executemany(sql, ciga_ls)
connection.commit()

In [ ]:
sql = ''
sql = '''
SELECT * FROM ciga;
'''

In [ ]:
cursor.execute(sql)

2380

In [ ]:
result = cursor.fetchall()
result

[{'AMOUNT': Decimal('236480540.000'),
  'CIGA_SUM': Decimal('254176377000.000'),
  'C_PRC': Decimal('9100.000'),
  'STK_CD': '000020',
  'STOCK_CNT': Decimal('27931470.000'),
  'VOL': Decimal('26013.000')},
 {'AMOUNT': Decimal('22041475700.000'),
  'CIGA_SUM': Decimal('10545600000000.000'),
  'C_PRC': Decimal('15600.000'),
  'STK_CD': '000030',
  'STOCK_CNT': Decimal('676000000.000'),
  'VOL': Decimal('1406627.000')},
 {'AMOUNT': Decimal('527636647.000'),
  'CIGA_SUM': Decimal('64491045000.000'),
  'C_PRC': Decimal('500.000'),
  'STK_CD': '000040',
  'STOCK_CNT': Decimal('128982090.000'),
  'VOL': Decimal('1082604.000')},
 {'AMOUNT': Decimal('86997500.000'),
  'CIGA_SUM': Decimal('296084916000.000'),
  'C_PRC': Decimal('10800.000'),
  'STK_CD': '000050',
  'STOCK_CNT': Decimal('27415270.000'),
  'VOL': Decimal('8070.000')},
 {'AMOUNT': Decimal('2705559100.000'),
  'CIGA_SUM': Decimal('2483908000000.000'),
  'C_PRC': Decimal('21850.000'),
  'STK_CD': '000060',
  'STOCK_CNT': Decimal('11

In [ ]:
# result를 데이터 프레임으로 나타내기
ciga_df = pd.DataFrame(result)
ciga_df

,STK_CD,C_PRC,CIGA_SUM,VOL,AMOUNT,STOCK_CNT
0,000020,9100.000,254176377000.000,26013.000,236480540.000,27931470.000
1,000030,15600.000,10545600000000.000,1406627.000,22041475700.000,676000000.000
2,000040,500.000,64491045000.000,1082604.000,527636647.000,128982090.000
3,000050,10800.000,296084916000.000,8070.000,86997500.000,27415270.000
4,000060,21850.000,2483908000000.000,124766.000,2705559100.000,113680000.000
...,...,...,...,...,...,...
2375,950110,16300.000,401901602000.000,169161.000,2658818350.000,24656540.000
2376,950130,4285.000,115973225050.000,211014.000,883110360.000,27064930.000
2377,950140,6230.000,123776805180.000,56984.000,355399420.000,19867866.000
2378,950160,43150.000,2632907282500.000,558855.000,23669596850.000,61017550.000


In [ ]:
# 2. MySQL 구문을 활용하여 종목 선정 기준 데이터 프레임 만들기

In [ ]:
# sql 모드 변경 : ONLY_FULL_GROUP_BY 제외
sql = ''
sql = '''
SET SESSION sql_mode = 'STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION';
'''

In [ ]:
cursor.execute(sql)

0

In [ ]:
sql = ''
sql = '''
SELECT   J1.STK_CD, J1.STK_NM
		, ROUND( (J2.NET_INCOME / J1.NET_INCOME) * 100, 1 ) 부채비율
		, ROUND( (J4.NET_INCOME / J3.NET_INCOME) * 100, 1 ) 유동비율
        , ROUND( (F1.FIN_ITM_VAL / 1e8), 1 ) `당기순이익(억)`
        , ROUND( (F1.FIN_ITM_VAL / J1.Net_INCOME) * 100 ,1 ) ROE
        , ROUND( (C2.CIGA_SUM / F1.FIN_ITM_VAL), 1 ) PER
        , ROUND( SUM(H1.VOL) / 1e8 , 1 ) `거래량합(억)`
FROM jaemo J1
	INNER JOIN finance_y F1 ON (J1.STK_CD = F1.STK_CD)
	INNER JOIN ciga C2 ON (F1.STK_CD = C2.STK_CD)
    INNER JOIN history_ym H1 ON(C2.STK_CD = H1.STK_CD)
	INNER JOIN jaemo J2 ON (J2.SET_DT = J1.SET_DT AND J2.STK_NM = J1.STK_NM)
	,jaemo J3
	INNER JOIN jaemo J4 ON (J4.SET_DT = J3.SET_DT AND J4.STK_NM = J3.STK_NM)
WHERE J1.SET_DT = '2018-12-31'
AND J1.SET_DT = J3.SET_DT AND J1.STK_NM = J3.STK_NM
AND J1.ITEM_NM = "자본총계" AND J2.ITEM_NM = '부채총계'
AND J3.ITEM_NM = "유동부채" AND J4.ITEM_NM = '유동자산'
AND F1.FIN_ITM_NM = '당기순이익'
AND F1.YY = '2018'
AND H1.YM BETWEEN '201801' AND '201812'
GROUP BY J1.STK_CD
HAVING 부채비율 < 100 AND 유동비율 >200 AND ROE >= 5 AND PER < 10
AND `당기순이익(억)` >= 535.2
# 중첩조건을 통과한 종목중 30번째 당기순이익이 535.2억이므로
# 당기순이익이 535.2억 이상인 종목들의 거래량을 살펴본다.
ORDER BY `거래량합(억)` DESC;
'''

In [ ]:
cursor.execute(sql)

29

In [ ]:
res = cursor.fetchall()
res

[{'PER': Decimal('5.2'),
  'ROE': Decimal('17.9'),
  'STK_CD': '005930',
  'STK_NM': '삼성전자',
  '거래량합(억)': 30.6,
  '당기순이익(억)': 443448.6,
  '부채비율': Decimal('37.0'),
  '유동비율': Decimal('252.9')},
 {'PER': Decimal('3.1'),
  'ROE': Decimal('26.3'),
  'STK_CD': '121800',
  'STK_NM': '비덴트',
  '거래량합(억)': 2.8,
  '당기순이익(억)': 638.4,
  '부채비율': Decimal('16.3'),
  '유동비율': Decimal('738.3')},
 {'PER': Decimal('1.1'),
  'ROE': Decimal('89.1'),
  'STK_CD': '035620',
  'STK_NM': '바른손이앤에이',
  '거래량합(억)': 2.3,
  '당기순이익(억)': 808.7,
  '부채비율': Decimal('25.6'),
  '유동비율': Decimal('263.9')},
 {'PER': Decimal('5.5'),
  'ROE': Decimal('15.5'),
  'STK_CD': '000990',
  'STK_NM': 'DB하이텍',
  '거래량합(억)': 1.2,
  '당기순이익(억)': 868.3,
  '부채비율': Decimal('91.0'),
  '유동비율': Decimal('244.5')},
 {'PER': Decimal('8.3'),
  'ROE': Decimal('35.2'),
  'STK_CD': '003000',
  'STK_NM': '부광약품',
  '거래량합(억)': 1.1,
  '당기순이익(억)': 1456.6,
  '부채비율': Decimal('19.4'),
  '유동비율': Decimal('579.5')},
 {'PER': Decimal('9.6'),
  'ROE': Decimal('23.5'),
 

In [ ]:
df = pd.DataFrame(res)
df

,STK_CD,STK_NM,부채비율,유동비율,당기순이익(억),ROE,PER,거래량합(억)
0,005930,삼성전자,37.0,252.9,443448.6,17.9,5.2,30.6
1,121800,비덴트,16.3,738.3,638.4,26.3,3.1,2.8
2,035620,바른손이앤에이,25.6,263.9,808.7,89.1,1.1,2.3
3,000990,DB하이텍,91.0,244.5,868.3,15.5,5.5,1.2
4,003000,부광약품,19.4,579.5,1456.6,35.2,8.3,1.1
5,240810,원익IPS,38.8,267.6,868.6,23.5,9.6,0.7
6,031980,피에스케이,17.8,535.2,535.2,19.7,4.7,0.6
7,012330,현대모비스,40.3,239.3,18882.1,6.1,9.8,0.6
8,267270,현대건설기계,99.4,233.2,1410.6,9.0,6.0,0.5
9,051600,한전KPS,30.2,235.3,1613.3,16.4,9.3,0.5
